# Step 1 | Collect track data by music genre

The main objective of this notebook is to retrieve **generic features** (e.g. *popularity*, *artist*) as well as the main **11 Spotify audio features** (i.e. *danceability*, *energy*, *key*, *loudness*, *mode*, *speechiness*, *acousticness*, *instrumentalness*, *liveness*, *valence*, *tempo*) characterizing a **track**.

The scope of this model will be limited to the music genres that, in line with the study on [Individual Differences in Musical Taste](https://www.jstor.org/stable/10.5406/amerjpsyc.123.2.0199), can be mapped to different personalities or moods.

It's important to note that in the Spotify API genres are attributed to artists and not playlists. Therefore, in order to obtain a list of tracks belonging to a specific music genre, we need to start from the artist. It's clear that the representative music genre of an artist could be both varied or even change over time. For the purpose of simplifying the analysis for this initial prototype, we will attribute only one genre to each artist and we will assume that every album and track belonging to that artist will be characterized by the same music genre.

----

The **algorithm** will be developed according to three main steps:
1. Get a list of **<mark>artists<mark/>** belonging to the music genres in scope
2. Retrieve a list of **<mark>albums<mark/>** for the artists identified in the previous step
3. Retrieve the list of **<mark>tracks<mark/>** included in the albums identified in pt 2 above

----

In [1268]:
# imports
import json
import spotipy
import spotipy.util as util
import pandas as pd
import pickle
from spotipy.oauth2 import SpotifyClientCredentials

In [41]:
# retrieve client id and client secret from txt file saved locally
file = '/Users/valentinarizzati/Desktop/GitStuff/Projects/Engineering-Spotify/final/spotify_creds.txt'

with open(file,'r') as f: 
    f = f.read().splitlines()
    cid = f[0].split(':')[1]
    secret = f[1].split(':')[1]

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

# initiate spotipy client
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Initial EDA

In [56]:
# function to obtain artist's genre
def get_artist_genre(artist):
    result = sp.search(artist)
    track = result['tracks']['items'][0]

    artist = sp.artist(track['artists'][0]['external_urls']['spotify'])
    print("artist genres:", artist["genres"])

Let's now do some spotipy testing across a few music genres in Spotify.

In [57]:
# expect jazz/soul
get_artist_genre('Nina Simone')

artist genres: ['jazz blues', 'soul', 'soul jazz', 'torch song', 'vocal jazz']


In [58]:
# expect rock/metal
get_artist_genre('Nirvana')

artist genres: ['alternative rock', 'grunge', 'permanent wave', 'rock']


In [59]:
# expect rap/hip hop
get_artist_genre('2Pac')

artist genres: ['g funk', 'gangster rap', 'hip hop', 'rap', 'west coast rap']


In [60]:
# expect pop
get_artist_genre('Sia')

artist genres: ['australian dance', 'australian pop', 'pop']


In [61]:
# expect classical
get_artist_genre('Mozart')

artist genres: ['classical', 'classical era']


In [62]:
# expect dance/house
get_artist_genre('Avicii')

artist genres: ['dance pop', 'edm', 'pop', 'pop dance']


In [67]:
# expect electronic
get_artist_genre('Gramatik')

artist genres: ['downtempo', 'jazz boom bap', 'jazztronica', 'livetronica', 'nu jazz', 'slovenian electronic', 'trip hop']


In [68]:
# expect electronic
get_artist_genre('De Hofnar')

artist genres: ['tropical house']


In [69]:
# expect electronic
get_artist_genre('Moderat')

artist genres: ['environmental']


In [70]:
# expect electronic
get_artist_genre('Max Manie')

artist genres: ['deep tropical house', 'tropical house']


In [71]:
# expect pop
get_artist_genre('Billie Eilish')

artist genres: ['electropop', 'pop']


----

As we see from the testing above, **genre labeling is definitely not a perfect science** on the Spotify platform. Particularly, we notice that:<br/>
1. Most artists have more than one genre associated with their music.
2. A few artists present genres that are not in accordance with the popular definition. 
    - This is especially true for the music genre *electronic*, that is commonly identified in Spotify as *tropical house* or *environmental*.
    
----

## Retrieve tracks across 15 music genres

## 1. Artists by Genre

As a first step, we will find a list of artists belonging to the main music genres mentioned in the study. For this purpose, we will create a function that will streamline and automate the pipeline going forward. 

In [262]:
# define list of music genres we can map to moods/personalities
seed_genres = ['pop',
               'rap',
               'r&b', 
               'country',
               'rock',
               'metal', 
               'indie',
               'dance',  
               'edm', 
               'classical',
               'jazz',
               'blues',
               'soul',
               'electronic',
               'house'
              ]

In [263]:
len(seed_genres)

15

In [268]:
limit = 50

artists_one_genre = {} # mapping artists to a single genre
artists = {} # mapping artists to a genre for multiple genres

def get_artists_by_genre(genre):
    '''
    Given a music genre, return a list of 500
    artists that are characterized by that genre.
    Then attribute only one genre to a specific artist.
    '''
    for i in range(0,500,50): # offset set to 500 because we want to get 500 artists per genre
        artist_results = sp.search(q='genre:'+str(genre), type='artist',limit=limit,offset=i) 
        for j in range(0,limit):
            name = artist_results['artists']['items'][j]['name']
            artists[name] = genre
            artists_one_genre[name] = genre
            
    pass

def get_all_artists(genre_list):
    '''
    Given a list of genres, return a list of artists by genre.
    Then attribute only one genre to a specific artist.
    Save results to a dictionary.
    '''
    for genre in genre_list:
        get_artists_by_genre(genre)
        print("Pulled {0} artists from the {1} genre".format(len(artists_one_genre), genre))
        artists_one_genre.clear()
    
    
    print("\nPulled a total of {0} artists across {1} genres".format(len(artists), len(seed_genres)))
    return artists

In [269]:
get_all_artists(seed_genres)

Pulled 499 artists from the pop genre
Pulled 499 artists from the rap genre
Pulled 500 artists from the r&b genre
Pulled 500 artists from the country genre
Pulled 499 artists from the rock genre
Pulled 500 artists from the metal genre
Pulled 500 artists from the indie genre
Pulled 500 artists from the dance genre
Pulled 499 artists from the edm genre
Pulled 498 artists from the classical genre
Pulled 499 artists from the jazz genre
Pulled 500 artists from the blues genre
Pulled 500 artists from the soul genre
Pulled 500 artists from the electronic genre
Pulled 500 artists from the house genre

Pulled a total of 5267 artists across 15 genres


{'Drake': 'rap',
 'Olivia Rodrigo': 'pop',
 'Doja Cat': 'dance',
 'Taylor Swift': 'pop',
 'Justin Bieber': 'pop',
 'The Weeknd': 'r&b',
 'Ariana Grande': 'dance',
 'BTS': 'pop',
 'Nicki Minaj': 'dance',
 'Lil Wayne': 'rap',
 'Chris Brown': 'dance',
 'Dua Lipa': 'dance',
 'SZA': 'r&b',
 'Bruno Mars': 'dance',
 'Khalid': 'pop',
 'Megan Thee Stallion': 'rap',
 'Rihanna': 'dance',
 'Future': 'rap',
 'Billie Eilish': 'pop',
 'Ed Sheeran': 'pop',
 'Young Thug': 'rap',
 'Ty Dolla $ign': 'soul',
 'Migos': 'rap',
 'Machine Gun Kelly': 'rap',
 'Maroon 5': 'rock',
 'Cardi B': 'dance',
 'blackbear': 'rock',
 'Halsey': 'dance',
 'Marshmello': 'house',
 'Kali Uchis': 'pop',
 'Wiz Khalifa': 'dance',
 'Beyoncé': 'dance',
 'Rauw Alejandro': 'pop',
 'Giveon': 'r&b',
 'A Boogie Wit da Hoodie': 'rap',
 'Harry Styles': 'pop',
 'Miley Cyrus': 'dance',
 'Lady Gaga': 'dance',
 'NF': 'rap',
 'Glass Animals': 'indie',
 'Florida Georgia Line': 'rock',
 'Ozuna': 'pop',
 'JAY-Z': 'rap',
 'One Direction': 'pop',
 '

In [270]:
# convert artists dictionary into a df
artists_df = pd.DataFrame(list(artists.items()),columns = ['artist','genre'])

In [271]:
artists_df.head()

,artist,genre
0,Drake,rap
1,Olivia Rodrigo,pop
2,Doja Cat,dance
3,Taylor Swift,pop
4,Justin Bieber,pop


In [274]:
artists_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5267 entries, 0 to 5266
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  5267 non-null   object
 1   genre   5267 non-null   object
dtypes: object(2)
memory usage: 82.4+ KB


In [272]:
# check for duplicate artists
duplicate_artists = artists_df[artists_df.duplicated()]

In [273]:
duplicate_artists

,artist,genre


In [280]:
# double check with different method
artists_df['artist'].is_unique

True

----

There is no duplicate artist in the dataframe and we have a total of **5267 artists** across **15 music genres**. 

----

## 2. Albums by Artist

We'll now pull the albums created by the 5267 seed artists.

In [517]:
limit = 5

albums_all = []
album_one_artist_df = pd.DataFrame(columns =['album_name', 'album_uri', 'num_tracks','artist_name']) # empty df
albums_all_df = pd.DataFrame(columns=['album_name','album_uri','num_tracks','artist_name']) # empty df

def get_albums_by_artist(artist):
    '''
    Given an artist, return a list of albums released
    by that artist.
    Retrieve album name, album uri, number of tracks,
    and artist name.
    Save results to a dataframe.
    '''
    for i in range(0,10,5):
        albums_results = sp.search(q='artist:'+str(artist), type='album',limit=limit,offset=i) 
        for j in range(0,len(albums_results['albums']['items'])-1): 
            album_name = albums_results['albums']['items'][j]['name']
            album_uri = albums_results['albums']['items'][j]['uri']
            album_tracks_num = albums_results['albums']['items'][j]['total_tracks']
            artist_name = albums_results['albums']['items'][j]['artists'][0]['name']
            album_one = [album_name, album_uri, album_tracks_num, artist_name]
            albums_all.append(album_one)
            
    album_temp = pd.DataFrame(albums_all, columns =['album_name', 'album_uri', 'num_tracks','artist_name'])    
    global album_one_artist_df
    album_one_artist_df = pd.concat([album_one_artist_df,album_temp])
    
    pass

def get_albums_all_artists(artists_obj):
    '''
    Given an object containint a list of artists, 
    return a list of albums released by those artist.
    Retrieve album name, album uri, number of tracks,
    and artist name.
    Save results to a dataframe.
    '''
    for artist in artists_obj:
        try:
            get_albums_by_artist(artist)
            global album_one_artist_df
            global albums_all_df
            albums_all_df = pd.concat([albums_all_df,album_one_artist_df.reset_index(drop=True)])
            album_one_artist_df = album_one_artist_df.iloc[0:0]
        except:
            pass
    
    return albums_all_df

In [507]:
# transform artist dictionary keys into a list including all the artists' names
artists_list = list(artists.keys())

In [518]:
# retrieve all the albums from the selected artists
get_albums_all_artists(artists_list)

,album_name,album_uri,num_tracks,artist_name
0,Scary Hours 2,spotify:album:5LuoozUhs2pl3glZeAJl89,3,Drake
1,Scorpion,spotify:album:1ATL5GLyefJaxhQzSPVrLX,25,Drake
2,Views,spotify:album:40GMAhriYJRO1rsY4YdrZb,20,Drake
3,More Life,spotify:album:1lXY618HWkwYKJWBRYR4MK,22,Drake
4,Dark Lane Demo Tapes,spotify:album:6OQ9gBfg5EXeNAEwGSs6jK,14,Drake
...,...,...,...,...
40262,Mother,spotify:album:28IlwCwIt69Xh627BwUC8F,14,In This Moment
40263,Emerald Isles,spotify:album:1KjGAHeyl9kHti8Z6ZT1pd,1,Moments of Clarity
40264,Panorama,spotify:album:4ZaeYUM5vgvkpexijZ3w2r,13,Møme
40265,Mindful Sleep,spotify:album:2kFyWqR5s7lvq8uNYGx6mj,3,Moments of Clarity


In [524]:
# reset index
albums_all_df = albums_all_df.reset_index(drop=True)

In [525]:
albums_all_df

,album_name,album_uri,num_tracks,artist_name
0,Scary Hours 2,spotify:album:5LuoozUhs2pl3glZeAJl89,3,Drake
1,Scorpion,spotify:album:1ATL5GLyefJaxhQzSPVrLX,25,Drake
2,Views,spotify:album:40GMAhriYJRO1rsY4YdrZb,20,Drake
3,More Life,spotify:album:1lXY618HWkwYKJWBRYR4MK,22,Drake
4,Dark Lane Demo Tapes,spotify:album:6OQ9gBfg5EXeNAEwGSs6jK,14,Drake
...,...,...,...,...
107019610,Mother,spotify:album:28IlwCwIt69Xh627BwUC8F,14,In This Moment
107019611,Emerald Isles,spotify:album:1KjGAHeyl9kHti8Z6ZT1pd,1,Moments of Clarity
107019612,Panorama,spotify:album:4ZaeYUM5vgvkpexijZ3w2r,13,Møme
107019613,Mindful Sleep,spotify:album:2kFyWqR5s7lvq8uNYGx6mj,3,Moments of Clarity


> More than 100 million albums seem quite a large number for 5267 unique artists. My intuition leads me to think that **duplicate albums** have been pulled from the API.

In [527]:
# check unique number of artists
len(pd.unique(albums_all_df['album_name']))

35135

In [528]:
# check unique number of artists
len(pd.unique(albums_all_df['album_uri']))

39225

> A URI (Uniform Resource Indicator) is a unique identifier of an album. Looking at the unique URIs it's clear that we need to eliminate duplicate albums from the dataframe.

In [526]:
# check unique number of artists
len(pd.unique(albums_all_df['artist_name']))

6745

> Interestingly, we also notice that there is a discrepancy between the 6745 resulting **number of unique artists** and the 5267 unique artists we started with. 

We will start by first investigating the artists discrepancy issue and then by eliminating duplicate albumns.

### Discrepancy in number of artists

Let's first check if the seed number of artists we started with (5267 unique artists) is a subset of the 6745 we find in the albumns dataframe. 

In [549]:
# function to check difference between sets of artists
def check_artists_subset(artist_list_1,artist_list_2):
    artist_set_1 = set(artist_list_1)
    artist_set_2 = set(artist_list_2)
    print("There are {0} unique artists in the initial set".format(len(artist_set_1)))
    print("\nThere are {0} unique artists in the set of artists derived from pulling the albums".format(len(artist_set_2)))
    if artist_set_1.issubset(artist_set_2) is True:
        print("\nSeed set of artists is a subset of set of artists derived from pulling the albums")
    else:
        print("\nThe seed artist set has {0} artists that are not included in the artist set derived from pulling the albums".format(len(artist_set_1-artist_set_2)))
        print("\nThe artist set derived from pulling the albums has {0} artists that are not included in the seed artist set".format(len(artist_set_2-artist_set_1)))      

In [550]:
# check difference between sets of artists
new_artists_list = albums_all_df['artist_name'].tolist()
check_artists_subset(artists_list,new_artists_list)

There are 5267 unique artists in the initial set

There are 6745 unique artists in the set of artists derived from pulling the albums

The seed artist set has 122 artists that are not included in the artist set derived from pulling the albums

The artist set derived from pulling the albums has 1600 artists that are not included in the seed artist set


> The artists that we're really interested in are the seed 5267 unique artists that we pulled in the first step of our analysis. Therefore, we'll focus on those. Since the great majority of them are present in the newly created album dataset (only 122 artists from the seed set are missing), we will proceed  with **filtering the albums dataset for only the 5267 unique artists present in the seed set**.

In [554]:
# filter for seed artists
albums_all_df = albums_all_df[albums_all_df['artist_name'].isin(artists_list)]

In [555]:
# check number of artists is the 5267 (seed artists) - 122
len(set(albums_all_df['artist_name'].tolist()))

5145

### Eliminate duplicate albums

Let's now eliminate duplicate albums for the 5145 artists in our set.

In [559]:
albums_all_df = albums_all_df.drop_duplicates(subset=['album_uri'])

In [560]:
albums_all_df

,album_name,album_uri,num_tracks,artist_name
0,Scary Hours 2,spotify:album:5LuoozUhs2pl3glZeAJl89,3,Drake
1,Scorpion,spotify:album:1ATL5GLyefJaxhQzSPVrLX,25,Drake
2,Views,spotify:album:40GMAhriYJRO1rsY4YdrZb,20,Drake
3,More Life,spotify:album:1lXY618HWkwYKJWBRYR4MK,22,Drake
4,Dark Lane Demo Tapes,spotify:album:6OQ9gBfg5EXeNAEwGSs6jK,14,Drake
...,...,...,...,...
106939088,Perfect Ghost,spotify:album:7BRmBGQ4t1kMYpNUgsv1OL,2,Myon
106979341,Boasty (feat. Idris Elba),spotify:album:7JoyVaXzxpgTPjkgB8mWk3,1,Wiley
106979346,"My One (feat. Tory Lanez, Kranium & Dappy)",spotify:album:1bE9v4U6xT3MDCh2SPn3jH,1,Wiley
107019612,Panorama,spotify:album:4ZaeYUM5vgvkpexijZ3w2r,13,Møme


In [561]:
# check unique number of artists
len(pd.unique(albums_all_df['album_uri']))

36978

## 3. Tracks by Album

In [1289]:
spotify = spotipy.Spotify(requests_timeout=100,retries=10)
from pandas.errors import MergeError

# function to extract tracks by album
def get_tracks_from_albums(list_of_albums_uri):
    
    '''
    For every album previously identified as 
    representative of a genre, find the list of tracks.
    
    Returns metadata for all tracks as well as music 
    features characterizing the tracks.
    
    Finally joins all data in one complete dataframe. 
    '''
    
    complete_tracks_df = pd.DataFrame() # empty df
    
    track_ids = []

    for album_uri in list_of_albums_uri:
        current_album_tracks = []
        album_id = album_uri.split(':')[2]
        album_results = sp.album(album_id) 

        for i in range(len(album_results['tracks']['items'])):
            track_id = album_results['tracks']['items'][i]['id']
            track_ids.append(track_id)
            current_album_tracks.append(track_id)

        track_meta={'id':[],'album_name':[], 'track_name':[], 
                       'artist_name':[],'explicit':[],'track_popularity':[]}
        
        for track_id in current_album_tracks:
            # get track's meta data
            meta = sp.track(track_id)
            
            # track id
            track_meta['id'].append(track_id)
            
            # album name
            album_name = meta['album']['name']
            track_meta['album_name']+=[album_name]

            # track name
            track_name = meta['name']
            track_meta['track_name']+=[track_name]

            # artist name
            s = ', '
            artist_name = s.join([artist['name'] for artist in meta['artists']])
            track_meta['artist_name']+=[artist_name]

            # explicit: lyrics could be considered offensive or unsuitable for children
            explicit = meta['explicit']
            track_meta['explicit'].append(explicit)

            # track popularity
            track_popularity = meta['popularity']
            track_meta['track_popularity'].append(track_popularity)

        # build a track metadata df 
        track_meta_df = pd.DataFrame.from_dict(track_meta) 

        # find track features and aggregate them in a df
        track_features = sp.audio_features(track_meta_df['id'])
        #print(len(track_features))
        try:
            track_features_df = pd.DataFrame.from_dict(track_features)
        except:
            pass
             
        try:
            # combine metadata and track features dataframes into one large dataframe
            tracks_df = track_meta_df.merge(track_features_df)
        except MergeError:
            pass
        
        complete_tracks_df = pd.concat([complete_tracks_df,tracks_df])
        
    return complete_tracks_df

In [1290]:
# convert album_uri col into a list of uris
albums_uri_list = albums_all_df['album_uri'].tolist()

In [1253]:
len(albums_uri_list)

36978

In [1291]:
# tracks_chunk_1 = get_tracks_from_albums(albums_uri_list[:2000])
# tracks_chunk_2 = get_tracks_from_albums(albums_uri_list[2000:2500])
# tracks_chunk_3 = get_tracks_from_albums(albums_uri_list[2500:5000])
# tracks_chunk_4 = get_tracks_from_albums(albums_uri_list[5000:7500])
# tracks_chunk_5 = get_tracks_from_albums(albums_uri_list[7500:10000])
# tracks_chunk_6 = get_tracks_from_albums(albums_uri_list[20000:22500])
tracks_chunk_7 = get_tracks_from_albums(albums_uri_list[22500:25000])

> **Note**: while trying to retrieve all the track data in one go, I keep running in a *timeout error*. After multiple tests on the code, I have decided to both increase the *requests_timeout* parameter to 100 and to pull the data in **<mark>chunks<mark/>**, running the process in parallel on **<mark>Jupyter Lab<mark/>** and **<mark>Google colab<mark/>** with GPU and high-RAM.

In [1269]:
# Store list in pickle object so we can pull track data simultaneously on jupyter lab and colab without incurring in timeout error
with open('albums_uri_list.pkl', 'wb') as f:
    pickle.dump(albums_uri_list, f)

In [1295]:
# upload colab chunk of track data
tracks_chunk_colab = pd.read_pickle('/Users/valentinarizzati/Desktop/GitStuff/Projects/Engineering-Spotify/final/track_colab.pkl')

In [1299]:
# concatenate all chunks in one final df
tracks_all_df = pd.concat([tracks_chunk_1,tracks_chunk_2,
                           tracks_chunk_3,tracks_chunk_4,
                           tracks_chunk_5, tracks_chunk_6,
                           tracks_chunk_7,tracks_chunk_colab])

In [1300]:
tracks_all_df

,id,album_name,track_name,artist_name,explicit,track_popularity,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,3aQem4jVGdhtg116TmJnHz,Scary Hours 2,What’s Next,Drake,True,82,0.781,0.594,0,-6.959,...,0.000000,0.1620,0.0628,129.895,audio_features,spotify:track:3aQem4jVGdhtg116TmJnHz,https://api.spotify.com/v1/tracks/3aQem4jVGdht...,https://api.spotify.com/v1/audio-analysis/3aQe...,178154,4
1,65OVbaJR5O1RmwOQx0875b,Scary Hours 2,Wants and Needs (feat. Lil Baby),"Drake, Lil Baby",True,89,0.578,0.449,1,-6.349,...,0.000002,0.1190,0.1000,136.006,audio_features,spotify:track:65OVbaJR5O1RmwOQx0875b,https://api.spotify.com/v1/tracks/65OVbaJR5O1R...,https://api.spotify.com/v1/audio-analysis/65OV...,192956,4
2,4FRW5Nza1Ym91BGV4nFWXI,Scary Hours 2,Lemon Pepper Freestyle (feat. Rick Ross),"Drake, Rick Ross",True,77,0.770,0.637,1,-5.530,...,0.000000,0.1710,0.4310,94.966,audio_features,spotify:track:4FRW5Nza1Ym91BGV4nFWXI,https://api.spotify.com/v1/tracks/4FRW5Nza1Ym9...,https://api.spotify.com/v1/audio-analysis/4FRW...,383037,4
0,2yg9UN4eo5eMVJ7OB4RWj3,Scorpion,Survival,Drake,True,59,0.555,0.596,9,-9.090,...,0.000000,0.1230,0.1450,77.324,audio_features,spotify:track:2yg9UN4eo5eMVJ7OB4RWj3,https://api.spotify.com/v1/tracks/2yg9UN4eo5eM...,https://api.spotify.com/v1/audio-analysis/2yg9...,136187,4
1,0TlLq3lA83rQOYtrqBqSct,Scorpion,Nonstop,Drake,True,77,0.912,0.412,7,-8.074,...,0.012600,0.1040,0.4230,154.983,audio_features,spotify:track:0TlLq3lA83rQOYtrqBqSct,https://api.spotify.com/v1/tracks/0TlLq3lA83rQ...,https://api.spotify.com/v1/audio-analysis/0TlL...,238614,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,6VP69eiW8k8OVUjS2lBYKy,Rise of the Blood Legion- The Best of (Chapter 1),Adrenalize,In This Moment,False,30,0.443,0.960,6,-3.933,...,0.000000,0.1310,0.4320,132.190,audio_features,spotify:track:6VP69eiW8k8OVUjS2lBYKy,https://api.spotify.com/v1/tracks/6VP69eiW8k8O...,https://api.spotify.com/v1/audio-analysis/6VP6...,255653,4
14,3GrFAJiT7FWRBKuLqVHy3b,Rise of the Blood Legion- The Best of (Chapter 1),It is Written,In This Moment,True,19,0.299,0.762,1,-15.699,...,0.377000,0.6070,0.1570,80.819,audio_features,spotify:track:3GrFAJiT7FWRBKuLqVHy3b,https://api.spotify.com/v1/tracks/3GrFAJiT7FWR...,https://api.spotify.com/v1/audio-analysis/3GrF...,30227,4
15,326QxhmInOvbgbaNzeHxRz,Rise of the Blood Legion- The Best of (Chapter 1),Burn,In This Moment,False,27,0.358,0.819,4,-5.256,...,0.003870,0.1050,0.0452,146.093,audio_features,spotify:track:326QxhmInOvbgbaNzeHxRz,https://api.spotify.com/v1/tracks/326QxhmInOvb...,https://api.spotify.com/v1/audio-analysis/326Q...,284693,4
16,5hamZeMqIFSZbEgQ2kQ9IN,Rise of the Blood Legion- The Best of (Chapter 1),Whore,In This Moment,True,33,0.474,0.884,6,-3.845,...,0.000000,0.0783,0.5240,179.559,audio_features,spotify:track:5hamZeMqIFSZbEgQ2kQ9IN,https://api.spotify.com/v1/tracks/5hamZeMqIFSZ...,https://api.spotify.com/v1/audio-analysis/5ham...,245960,4


In [1319]:
# check concatenation was executed correctly

# list of track chunks pulled on jupyter lab
track_chunks = [tracks_chunk_1, tracks_chunk_2, tracks_chunk_3,
                tracks_chunk_4, tracks_chunk_5, tracks_chunk_6,
                tracks_chunk_7]
len_chunks = 0

for i in track_chunks:
    len_chunks += len(i)

len_chunks = len_chunks + len(tracks_chunk_colab) # add len of tracks data pulled on colab

if len(tracks_all_df) == len_chunks:
    print("Chunks concatenation executed correctly")
else:
    print("Not all records have been included in the concatenated dataframe")

Chunks concatenation executed correctly


In [1320]:
# convert milliseconds to mins
# duration_ms: The duration of the track in milliseconds.
# 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
tracks_all_df['duration_mins'] = tracks_all_df['duration_ms']/60000

In [1323]:
# reset index
tracks_all_df = tracks_all_df.reset_index(drop=True)

In [1324]:
tracks_all_df.head()

,id,album_name,track_name,artist_name,explicit,track_popularity,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,duration_mins
0,3aQem4jVGdhtg116TmJnHz,Scary Hours 2,What’s Next,Drake,True,82,0.781,0.594,0,-6.959,...,0.162,0.0628,129.895,audio_features,spotify:track:3aQem4jVGdhtg116TmJnHz,https://api.spotify.com/v1/tracks/3aQem4jVGdht...,https://api.spotify.com/v1/audio-analysis/3aQe...,178154,4,2.969233
1,65OVbaJR5O1RmwOQx0875b,Scary Hours 2,Wants and Needs (feat. Lil Baby),"Drake, Lil Baby",True,89,0.578,0.449,1,-6.349,...,0.119,0.1000,136.006,audio_features,spotify:track:65OVbaJR5O1RmwOQx0875b,https://api.spotify.com/v1/tracks/65OVbaJR5O1R...,https://api.spotify.com/v1/audio-analysis/65OV...,192956,4,3.215933
2,4FRW5Nza1Ym91BGV4nFWXI,Scary Hours 2,Lemon Pepper Freestyle (feat. Rick Ross),"Drake, Rick Ross",True,77,0.770,0.637,1,-5.530,...,0.171,0.4310,94.966,audio_features,spotify:track:4FRW5Nza1Ym91BGV4nFWXI,https://api.spotify.com/v1/tracks/4FRW5Nza1Ym9...,https://api.spotify.com/v1/audio-analysis/4FRW...,383037,4,6.383950
3,2yg9UN4eo5eMVJ7OB4RWj3,Scorpion,Survival,Drake,True,59,0.555,0.596,9,-9.090,...,0.123,0.1450,77.324,audio_features,spotify:track:2yg9UN4eo5eMVJ7OB4RWj3,https://api.spotify.com/v1/tracks/2yg9UN4eo5eM...,https://api.spotify.com/v1/audio-analysis/2yg9...,136187,4,2.269783
4,0TlLq3lA83rQOYtrqBqSct,Scorpion,Nonstop,Drake,True,77,0.912,0.412,7,-8.074,...,0.104,0.4230,154.983,audio_features,spotify:track:0TlLq3lA83rQOYtrqBqSct,https://api.spotify.com/v1/tracks/0TlLq3lA83rQ...,https://api.spotify.com/v1/audio-analysis/0TlL...,238614,4,3.976900


In [1325]:
# check if there are any null values
tracks_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346636 entries, 0 to 346635
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                346636 non-null  object 
 1   album_name        346636 non-null  object 
 2   track_name        346636 non-null  object 
 3   artist_name       346636 non-null  object 
 4   explicit          346636 non-null  bool   
 5   track_popularity  346636 non-null  int64  
 6   danceability      346636 non-null  float64
 7   energy            346636 non-null  float64
 8   key               346636 non-null  int64  
 9   loudness          346636 non-null  float64
 10  mode              346636 non-null  int64  
 11  speechiness       346636 non-null  float64
 12  acousticness      346636 non-null  float64
 13  instrumentalness  346636 non-null  float64
 14  liveness          346636 non-null  float64
 15  valence           346636 non-null  float64
 16  tempo             34

> It looks like there are not null values. Let's double check with another method.

In [1326]:
tracks_all_df.isnull().values.any()

False

In [1327]:
tracks_all_df.isnull().sum()

id                  0
album_name          0
track_name          0
artist_name         0
explicit            0
track_popularity    0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
type                0
uri                 0
track_href          0
analysis_url        0
duration_ms         0
time_signature      0
duration_mins       0
dtype: int64

> Great! After double checking with a couple of methods, we can state that there aren't **any null values**. 

In [1328]:
# check unique tracks based on their id
len(pd.unique(tracks_all_df['id']))

346557

> By comparing the length of the df with the count of unique track ids, we notice that there are **duplicate tracks**. Let's eliminate them. 

In [1331]:
# drop duplicates
tracks_all_df = tracks_all_df.drop_duplicates(subset=['id'])

In [1332]:
len(tracks_all_df)

346557

> Great! Duplicate tracks have been eliminater and this is the final length of our tracks dataframe.

## Save tables to SQL db

We will now store the three tables *artists*, *albums* and *tracks* in a SQL db instance. This will allow us to store the data permanently on a local machine for use in future analyses.

In [1333]:
# initiate sqlalchemy and create engine
from sqlalchemy import create_engine
engine = create_engine('sqlite:////Users/valentinarizzati/Desktop/GitStuff/Projects/Engineering-Spotify/final/spotify.db', echo=False)

In [1334]:
# create tables
with engine.begin() as connection:
    # create artists table
    artists_df.to_sql('artists', con=connection, if_exists='append')
    # create albums table
    albums_all_df.to_sql('albums', con=connection, if_exists='append')
    # create tracks table
    tracks_all_df.to_sql('tracks', con=connection, if_exists='append')

In [1335]:
# check tables where created
engine.table_names()

['albums', 'artists', 'tracks']

In [1336]:
# check data was saved
pd.read_sql('SELECT * FROM artists;', engine)

,index,artist,genre
0,0,Drake,rap
1,1,Olivia Rodrigo,pop
2,2,Doja Cat,dance
3,3,Taylor Swift,pop
4,4,Justin Bieber,pop
...,...,...,...
5262,5262,Jiony,house
5263,5263,The Toxic Avenger,house
5264,5264,Myon,house
5265,5265,Wiley,house


In [1337]:
# check data was saved
pd.read_sql('SELECT * FROM albums;', engine)

,index,album_name,album_uri,num_tracks,artist_name
0,0,Scary Hours 2,spotify:album:5LuoozUhs2pl3glZeAJl89,3,Drake
1,1,Scorpion,spotify:album:1ATL5GLyefJaxhQzSPVrLX,25,Drake
2,2,Views,spotify:album:40GMAhriYJRO1rsY4YdrZb,20,Drake
3,3,More Life,spotify:album:1lXY618HWkwYKJWBRYR4MK,22,Drake
4,4,Dark Lane Demo Tapes,spotify:album:6OQ9gBfg5EXeNAEwGSs6jK,14,Drake
...,...,...,...,...,...
36973,106939088,Perfect Ghost,spotify:album:7BRmBGQ4t1kMYpNUgsv1OL,2,Myon
36974,106979341,Boasty (feat. Idris Elba),spotify:album:7JoyVaXzxpgTPjkgB8mWk3,1,Wiley
36975,106979346,"My One (feat. Tory Lanez, Kranium & Dappy)",spotify:album:1bE9v4U6xT3MDCh2SPn3jH,1,Wiley
36976,107019612,Panorama,spotify:album:4ZaeYUM5vgvkpexijZ3w2r,13,Møme


In [1338]:
# check data was saved
pd.read_sql('SELECT * FROM tracks;', engine)

,index,id,album_name,track_name,artist_name,explicit,track_popularity,danceability,energy,key,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,duration_mins
0,0,3aQem4jVGdhtg116TmJnHz,Scary Hours 2,What’s Next,Drake,1,82,0.781,0.594,0,...,0.1620,0.0628,129.895,audio_features,spotify:track:3aQem4jVGdhtg116TmJnHz,https://api.spotify.com/v1/tracks/3aQem4jVGdht...,https://api.spotify.com/v1/audio-analysis/3aQe...,178154,4,2.969233
1,1,65OVbaJR5O1RmwOQx0875b,Scary Hours 2,Wants and Needs (feat. Lil Baby),"Drake, Lil Baby",1,89,0.578,0.449,1,...,0.1190,0.1000,136.006,audio_features,spotify:track:65OVbaJR5O1RmwOQx0875b,https://api.spotify.com/v1/tracks/65OVbaJR5O1R...,https://api.spotify.com/v1/audio-analysis/65OV...,192956,4,3.215933
2,2,4FRW5Nza1Ym91BGV4nFWXI,Scary Hours 2,Lemon Pepper Freestyle (feat. Rick Ross),"Drake, Rick Ross",1,77,0.770,0.637,1,...,0.1710,0.4310,94.966,audio_features,spotify:track:4FRW5Nza1Ym91BGV4nFWXI,https://api.spotify.com/v1/tracks/4FRW5Nza1Ym9...,https://api.spotify.com/v1/audio-analysis/4FRW...,383037,4,6.383950
3,3,2yg9UN4eo5eMVJ7OB4RWj3,Scorpion,Survival,Drake,1,59,0.555,0.596,9,...,0.1230,0.1450,77.324,audio_features,spotify:track:2yg9UN4eo5eMVJ7OB4RWj3,https://api.spotify.com/v1/tracks/2yg9UN4eo5eM...,https://api.spotify.com/v1/audio-analysis/2yg9...,136187,4,2.269783
4,4,0TlLq3lA83rQOYtrqBqSct,Scorpion,Nonstop,Drake,1,77,0.912,0.412,7,...,0.1040,0.4230,154.983,audio_features,spotify:track:0TlLq3lA83rQOYtrqBqSct,https://api.spotify.com/v1/tracks/0TlLq3lA83rQ...,https://api.spotify.com/v1/audio-analysis/0TlL...,238614,4,3.976900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346552,346631,6VP69eiW8k8OVUjS2lBYKy,Rise of the Blood Legion- The Best of (Chapter 1),Adrenalize,In This Moment,0,30,0.443,0.960,6,...,0.1310,0.4320,132.190,audio_features,spotify:track:6VP69eiW8k8OVUjS2lBYKy,https://api.spotify.com/v1/tracks/6VP69eiW8k8O...,https://api.spotify.com/v1/audio-analysis/6VP6...,255653,4,4.260883
346553,346632,3GrFAJiT7FWRBKuLqVHy3b,Rise of the Blood Legion- The Best of (Chapter 1),It is Written,In This Moment,1,19,0.299,0.762,1,...,0.6070,0.1570,80.819,audio_features,spotify:track:3GrFAJiT7FWRBKuLqVHy3b,https://api.spotify.com/v1/tracks/3GrFAJiT7FWR...,https://api.spotify.com/v1/audio-analysis/3GrF...,30227,4,0.503783
346554,346633,326QxhmInOvbgbaNzeHxRz,Rise of the Blood Legion- The Best of (Chapter 1),Burn,In This Moment,0,27,0.358,0.819,4,...,0.1050,0.0452,146.093,audio_features,spotify:track:326QxhmInOvbgbaNzeHxRz,https://api.spotify.com/v1/tracks/326QxhmInOvb...,https://api.spotify.com/v1/audio-analysis/326Q...,284693,4,4.744883
346555,346634,5hamZeMqIFSZbEgQ2kQ9IN,Rise of the Blood Legion- The Best of (Chapter 1),Whore,In This Moment,1,33,0.474,0.884,6,...,0.0783,0.5240,179.559,audio_features,spotify:track:5hamZeMqIFSZbEgQ2kQ9IN,https://api.spotify.com/v1/tracks/5hamZeMqIFSZ...,https://api.spotify.com/v1/audio-analysis/5ham...,245960,4,4.099333


> This looks great. All data about artists, albumns and tracks was stored locally. 

----

By using a combination of Jupyter Lab and Google Colab to pull data from the Spotify API, we have now reached the **conclusion of the first step in our pipeline**.<br/>
We will now proceed with data preprocessing and modeling in **<mark>notebook 02<mark/>** of this series. 

----